In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('<path to your dataset>')
df.head()

## Exploratory Data Analysis

**1. Plot the distribution of the target variable: 'Grouping'**

In [ ]:
df['Grouping'].value_counts().plot(kind='bar')

In [ ]:
sns.barplot(data=df['Grouping'].value_counts(), orient='h')

In [ ]:
print(df['Age (years)'].value_counts())
sns.barplot(data=df['Age (years)'].value_counts(), orient='h')

In [ ]:
# Number of cases in each age group
# And then plot each cases group by Age (years)
sns.countplot(data=df, x='Grouping', hue='Age (years)')

In [ ]:
sns.countplot(data=df, x='Grouping', hue='BMI')

In [ ]:
sns.countplot(data=df, x='Grouping', hue='Highfat')

In [ ]:
df['Highfat'].value_counts()

In [ ]:
sns.countplot(data=df, x='Grouping', hue='Marital status')

In [ ]:
sns.barplot(data=df['Marital status'].value_counts(), orient='h')

## Decision Tree

In [ ]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
prep_data = enc.fit_transform(df)

In [ ]:
# Convert the new feature to dataframe
prep_data = pd.DataFrame(prep_data, columns=df.columns)
prep_data.head()

In [ ]:
X = prep_data.drop(columns='Grouping')
y = prep_data['Grouping']

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(
)
clf = clf.fit(X, y)

In [ ]:
tree.plot_tree(clf)

In [ ]:
import graphviz
dot_data = tree.export_graphviz(clf, out_file=None,
								feature_names=X.columns,
								class_names=df['Grouping'].unique(),
								filled=True, rounded=True,
								special_characters=True)
graph = graphviz.Source(dot_data)
graph 